# Lab | Data Structuring and Combining Data

## Challenge 1: Combining & Cleaning Data

In this challenge, we will be working with the customer data from an insurance company, as we did in the two previous labs. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

But this time, we got new data, which can be found in the following 2 CSV files located at the links below.

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv

Note that you'll need to clean and format the new data.

Observation:
- One option is to first combine the three datasets and then apply the cleaning function to the new combined dataset
- Another option would be to read the clean file you saved in the previous lab, and just clean the two new files and concatenate the three clean datasets

In [ ]:
import pandas as pd

urls = [
    "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv",
    "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv",
    "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv",
]

def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = (
        df.columns
          .str.strip()
          .str.lower()
          .str.replace(r"\s+", "_", regex=True)
    )
    df.rename(columns={"st": "state"}, inplace=True)
    return df

def standardize_values(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    if "gender" in df.columns and pd.api.types.is_string_dtype(df["gender"]):
        df["gender"] = (
            df["gender"].astype(str).str.strip().str.lower()
              .replace({"m":"M","male":"M","f":"F","female":"F","femal":"F"})
        )

    if "education" in df.columns and pd.api.types.is_string_dtype(df["education"]):
        df["education"] = (
            df["education"].astype(str).str.strip().str.title()
              .replace({"Bachelors":"Bachelor"})
        )

    if "state" in df.columns and pd.api.types.is_string_dtype(df["state"]):
        df["state"] = (
            df["state"].astype(str).str.strip().str.title()
              .replace({"Cali":"California","Az":"Arizona","Wa":"Washington"})
        )

    if "vehicle_class" in df.columns and pd.api.types.is_string_dtype(df["vehicle_class"]):
        df["vehicle_class"] = (
            df["vehicle_class"].astype(str).str.strip().str.title()
              .replace({"Luxury Suv":"Luxury","Luxury Car":"Luxury","Sports Car":"Luxury"})
        )

    if "customer_lifetime_value" in df.columns:
        df["customer_lifetime_value"] = (
            df["customer_lifetime_value"]
              .astype(str).str.replace("%","", regex=False).str.strip()
        )
        df["customer_lifetime_value"] = pd.to_numeric(df["customer_lifetime_value"], errors="coerce")

    if "number_of_open_complaints" in df.columns:
        def middle_num(x):
            if pd.isna(x): return pd.NA
            x = str(x)
            parts = x.split("/")
            if len(parts) >= 2 and parts[1].isdigit():
                return int(parts[1])
            try:
                return int(float(x))
            except:
                return pd.NA
        df["number_of_open_complaints"] = df["number_of_open_complaints"].apply(middle_num)

    return df

def cast_types(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for col in ["income","monthly_premium_auto","total_claim_amount","customer_lifetime_value","number_of_open_complaints"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    for col in ["effective_to_date","response_date","policy_start_date"]:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors="coerce")

    return df

def clean_df(df: pd.DataFrame) -> pd.DataFrame:
    df = clean_columns(df)
    df = standardize_values(df)
    df = cast_types(df)
    return df

frames = [pd.read_csv(u) for u in urls]
raw_combined = pd.concat(frames, axis=0, ignore_index=True)

df_clean = clean_df(raw_combined)

df_clean = df_clean.drop_duplicates().reset_index(drop=True)

print(df_clean.shape)
print(df_clean.dtypes.head(13))
df_clean.head(5)

# Challenge 2: Structuring Data

In this challenge, we will continue to work with customer data from an insurance company, but we will use a dataset with more columns, called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing data cleaning, formatting, and structuring.

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv"
df = pd.read_csv(url)

df.head(3)

In [7]:
pivot_sales = df.pivot_table(
    index='sales_channel',
    values='total_claim_amount',
    aggfunc='sum'
).round(2)

# Ordenar descendente por ingresos
pivot_sales = pivot_sales.sort_values(by='total_claim_amount', ascending=False)

print("Total revenue by Sales Channel:")
print(pivot_sales)

Total revenue by Sales Channel:
               total_claim_amount
sales_channel                    
Agent                  1810226.82
Branch                 1301204.00
Call Center             926600.82
Web                     706600.04


1. You work at the marketing department and you want to know which sales channel brought the most sales in terms of total revenue. Using pivot, create a summary table showing the total revenue for each sales channel (branch, call center, web, and mail).
Round the total revenue to 2 decimal points.  Analyze the resulting table to draw insights.

2. Create a pivot table that shows the average customer lifetime value per gender and education level. Analyze the resulting table to draw insights.

## Bonus

You work at the customer service department and you want to know which months had the highest number of complaints by policy type category. Create a summary table showing the number of complaints by policy type and month.
Show it in a long format table.

*In data analysis, a long format table is a way of structuring data in which each observation or measurement is stored in a separate row of the table. The key characteristic of a long format table is that each column represents a single variable, and each row represents a single observation of that variable.*

*More information about long and wide format tables here: https://www.statology.org/long-vs-wide-data/*

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv"
df = pd.read_csv(url)

df.head(3)

In [ ]:
df["effective_to_date"] = pd.to_datetime(df["effective_to_date"], errors="coerce")

df["month"] = df["effective_to_date"].dt.month_name()

df[["effective_to_date", "month"]].head()

In [ ]:
df["number_of_open_complaints"] = df["number_of_open_complaints"].fillna(0)

complaints_summary = (
    df.pivot_table(
        index="policy_type",
        columns="month",
        values="number_of_open_complaints",
        aggfunc="sum"
    )
)

complaints_summary.head()

In [ ]:
complaints_long = complaints_summary.reset_index().melt(
    id_vars="policy_type",
    var_name="month",
    value_name="total_complaints"
)

complaints_long.head(5)